In [22]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [23]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [24]:
num_classes = len(gloss_df.gloss.unique())

In [25]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [26]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

In [27]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(25),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomShortSideScale(min_size=224, max_size=256),
        CenterCropVideo(224),
    ]),
    ),
])

In [28]:
train_path = "/home/toghrul/SLR/data/train"
val_path = "/home/toghrul/SLR/data/val"
test_path = "/home/toghrul/SLR/data/test"

In [43]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.metrics import classification_report
import torchmetrics

In [55]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath="checkpoints", 
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


In [56]:
seed_everything(0)

trainer = Trainer(max_epochs=15,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=25,
                limit_val_batches=10,
                limit_test_batches=10,)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [57]:
# trainer.fit(model, ckpt_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt")
trainer.fit(model)

Restoring states from the checkpoint path at /home/toghrul/SLR/sign-lang/checkpoints/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | fc          | Linear             | 93.8 K
3 | metric      | MultilabelAccuracy | 0     
4 | criterion   | BCEWithLogitsLoss  | 0     
---------------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
7.776     Total estimated model params size (MB)
Restored all states from the checkpoint file at /home/toghrul/SLR/sign-lang/checkpoints/last.ckpt


Sanity Checking: 0it [00:00, ?it/s]

>>> Validation step No.0:
Loss: 0.030196867883205414
Accuracy: 0.9957265257835388
>>> Validation step No.1:
Loss: 0.03355490788817406
Accuracy: 0.9951923489570618


Training: 50it [00:00, ?it/s]

>>> Training step No.0:
Loss: 0.019229505211114883
Accuracy: 0.9957265257835388
>>> Training step No.1:
Loss: 0.0268651582300663
Accuracy: 0.9957265257835388
>>> Training step No.2:
Loss: 0.0164125245064497
Accuracy: 0.9957265257835388
>>> Training step No.3:
Loss: 0.02180781215429306
Accuracy: 0.9962607026100159
>>> Training step No.4:
Loss: 0.02651321515440941
Accuracy: 0.9957265257835388
>>> Training step No.5:
Loss: 0.025027191266417503
Accuracy: 0.995726466178894
>>> Training step No.6:
Loss: 0.02019244246184826
Accuracy: 0.9962607026100159
>>> Training step No.7:
Loss: 0.02203991636633873
Accuracy: 0.9962607026100159
>>> Training step No.8:
Loss: 0.02086511068046093
Accuracy: 0.9967949390411377
>>> Training step No.9:
Loss: 0.017803754657506943
Accuracy: 0.9962607622146606
>>> Training step No.10:
Loss: 0.02290123887360096
Accuracy: 0.9962607026100159
>>> Training step No.11:
Loss: 0.026080913841724396
Accuracy: 0.9962607622146606
>>> Training step No.12:
Loss: 0.0260566826909780

...

Epoch 11, global step 175: 'val_loss' was not in top 2


>>> Validation step No.11:
Loss: 0.03156976401805878
Accuracy: 0.9957265853881836
>>> Epoch end loss: 0.019999999552965164
>>> Training step No.25:
Loss: 0.023513279855251312
Accuracy: 0.9962607026100159
>>> Training step No.26:
Loss: 0.01939724199473858
Accuracy: 0.9957265853881836
>>> Training step No.27:
Loss: 0.02543318271636963
Accuracy: 0.9957265257835388
>>> Training step No.28:
Loss: 0.023127995431423187
Accuracy: 0.9957265257835388
>>> Training step No.29:
Loss: 0.02252042666077614
Accuracy: 0.9962607622146606
>>> Training step No.30:
Loss: 0.016421256586909294
Accuracy: 0.9957265257835388
>>> Training step No.31:
Loss: 0.02178790606558323
Accuracy: 0.9957265257835388
>>> Training step No.32:
Loss: 0.025772666558623314
Accuracy: 0.9957265257835388
>>> Training step No.33:
Loss: 0.017129290848970413
Accuracy: 0.9962607622146606
>>> Training step No.34:
Loss: 0.02106698416173458
Accuracy: 0.9967948198318481
>>> Training step No.35:
Loss: 0.027266182005405426
Accuracy: 0.99519234

Validation: 0it [00:00, ?it/s]

>>> Validation step No.12:
Loss: 0.03041241504251957
Accuracy: 0.9957265853881836
>>> Validation step No.13:
Loss: 0.027635902166366577
Accuracy: 0.9957265257835388
>>> Validation step No.14:
Loss: 0.029976190999150276
Accuracy: 0.9957265853881836
>>> Validation step No.15:
Loss: 0.03118453547358513
Accuracy: 0.9951923489570618


/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [18]:
model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt",
    hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_38/hparams.yaml",
    map_location=None,
)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [44]:
test_res = trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

>> Label: [190 135 103  72  50  68 171 130 168  31 108  34 124  77  13 108 108  68
 188 194 132 174  16 101  55  21  47  46  75 108 186   2  42  61  50 108
 108  55   8  14]
Pred: [120 166 166 120 120 166 166 166 120 120  27 120 120 166 166  27 166 166
 166   6 120 120   6 166 166 126 166  27 120 120 158 120 120 120 120  27
 166 166 120 120]


In [19]:
val_res = trainer.validate(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

>>> Validation step No.0:
Loss: 0.02925325557589531
Accuracy: 0.9951923489570618
>>> Validation step No.1:
Loss: 0.02883106842637062
Accuracy: 0.9957265257835388
>>> Validation step No.2:
Loss: 0.033470068126916885
Accuracy: 0.9957265257835388
>>> Validation step No.3:
Loss: 0.031304895877838135
Accuracy: 0.9951923489570618
>>> Validation step No.4:
Loss: 0.03123767301440239
Accuracy: 0.9957265853881836
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss           0.029999999329447746
       val_metric                   1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [21]:
a = torch.randn(8, 1)

a

(tensor([[-1.1373],
         [ 1.2810],
         [ 0.6021],
         [-0.2140],
         [-0.1674],
         [-0.0197],
         [ 0.5376],
         [-0.6672]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0]))

In [33]:
a = np.random.randn(8, 4)
a, a.argmax(axis=1)

(array([[-0.88778575, -1.98079647, -0.34791215,  0.15634897],
        [ 1.23029068,  1.20237985, -0.38732682, -0.30230275],
        [-1.04855297, -1.42001794, -1.70627019,  1.9507754 ],
        [-0.50965218, -0.4380743 , -1.25279536,  0.77749036],
        [-1.61389785, -0.21274028, -0.89546656,  0.3869025 ],
        [-0.51080514, -1.18063218, -0.02818223,  0.42833187],
        [ 0.06651722,  0.3024719 , -0.63432209, -0.36274117],
        [-0.67246045, -0.35955316, -0.81314628, -1.7262826 ]]),
 array([3, 0, 3, 3, 3, 3, 1, 1]))